In [11]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [12]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import numpy as np
from typing import Dict
import polars as pl

from src.common.AssetData import AssetData
from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.featureAlchemy.FeatureFinancialData import FeatureFinancialData


In [10]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_snp500_finanTo2011")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)
    
ticker = "NVDA"
asset = assetspl[ticker]
asset.financials_quarterly.select(["fiscalDateEnding",	"reportedDate",	"reportedEPS",	"estimatedEPS","surprise",	"surprisePercentage",	"reportTime"]).tail(5)

fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime
"datetime[ns, UTC]","datetime[ns, UTC]",f64,f64,f64,f64,str
2023-10-29 00:00:00 UTC,2023-11-21 00:00:00 UTC,0.402,0.336,0.066,19.6429,"""post-market"""
2024-01-28 00:00:00 UTC,2024-02-21 00:00:00 UTC,0.516,0.461,0.055,11.9306,"""post-market"""
2024-04-28 00:00:00 UTC,2024-05-22 00:00:00 UTC,0.612,0.559,0.053,9.4812,"""post-market"""
2024-07-28 00:00:00 UTC,2024-08-28 00:00:00 UTC,0.68,0.63,0.05,7.9365,"""post-market"""
2024-10-27 00:00:00 UTC,2024-11-20 00:00:00 UTC,0.81,0.74,0.07,9.4595,"""post-market"""


In [9]:
# assetspl[ticker].financials_quarterly is a dataframe with a date column "fiscalDateEnding". output the number of nan values in every column whose entries are in the year 2020
nan_counts_dict = {}

aCols = asset.financials_quarterly.columns
n_arr = np.zeros(len(aCols))
for ticker, asset in assetspl.items():
    if asset.financials_quarterly.is_empty():
        continue
    if asset.financials_quarterly["reportedEPS"].null_count() > 0:
        continue
    nan_counts_dict[ticker] = asset.financials_quarterly.filter(
        (asset.financials_quarterly["fiscalDateEnding"].dt.year() > 2019)
        ).null_count()
    n_arr += nan_counts_dict[ticker].row(0)

print(len(assetspl.keys()))
print(len(nan_counts_dict.keys()))
n_arr

1077
819


array([    0.,     0.,     0.,   661.,   661.,   712.,     0.,   101.,
          84.,    75.,   318.,   939.,    81.,   557., 11256.,    81.,
       10616.,   529.,  2919.,   511.,  2820.,  4899., 11488.,   150.,
          75.,   147.,  3710.,   112.,  1478.,   102.,   100.,    74.,
         136.,   109.,   122.,   132.,   133.,  1906.,  5245.,   125.,
         965.,  8346.,  1899.,  2835.,  3266.,  6476.,  6317.,  6540.,
         314.,  5340.,   109.,   146.,  5972.,  8138.,  5827.,  1061.,
         146.,  5233.,  1205.,  4234.,  6759.,   428.,   214.,  1838.,
         109.,  7211.,   398.,   240.,   108.,   120.,   113.,  4880.,
       14420.,  3450.,  2973.,   242.,   542.,  4801.,  4747.,  1481.,
         524.,   130.,  5312.,  3849.,  3458., 14895.,  2356.,  2438.,
       14592.,  9956.,  5757., 14384.,   831., 15034.,  5679., 14381.,
          98.])

In [6]:
aCols

['fiscalDateEnding',
 'reportedDate',
 'reportedEPS',
 'estimatedEPS',
 'surprise',
 'surprisePercentage',
 'reportTime',
 'reportedCurrency_x',
 'grossProfit',
 'totalRevenue',
 'costOfRevenue',
 'costofGoodsAndServicesSold',
 'operatingIncome',
 'sellingGeneralAndAdministrative',
 'researchAndDevelopment',
 'operatingExpenses',
 'investmentIncomeNet',
 'netInterestIncome',
 'interestIncome',
 'interestExpense',
 'nonInterestIncome',
 'otherNonOperatingIncome',
 'depreciation',
 'depreciationAndAmortization',
 'incomeBeforeTax',
 'incomeTaxExpense',
 'interestAndDebtExpense',
 'netIncomeFromContinuingOperations',
 'comprehensiveIncomeNetOfTax',
 'ebit',
 'ebitda',
 'netIncome_x',
 'reportedCurrency_y',
 'totalAssets',
 'totalCurrentAssets',
 'cashAndCashEquivalentsAtCarryingValue',
 'cashAndShortTermInvestments',
 'inventory',
 'currentNetReceivables',
 'totalNonCurrentAssets',
 'propertyPlantEquipment',
 'accumulatedDepreciationAmortizationPPE',
 'intangibleAssets',
 'intangibleAsset

In [4]:
fin_quar = asset.financials_quarterly
fin_ann = asset.financials_annually
asset.shareprice = asset.shareprice.with_columns(
    pl.col("Date").dt.replace_time_zone("UTC").alias("Date")
)

print((asset.shareprice["Date"].item(-1).tzinfo))
print(fin_quar["fiscalDateEnding"].item(-1).tzinfo)
print(fin_quar.columns)
ffd = FeatureFinancialData(asset, [1,2,3,5,9])
print(asset.shareprice.columns)
asShare = asset.shareprice
finQuar = ffd.fin_quar.select("reportedEPS")

UTC
UTC
['fiscalDateEnding', 'reportedDate', 'reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage', 'reportTime', 'reportedCurrency_x', 'grossProfit', 'totalRevenue', 'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome', 'sellingGeneralAndAdministrative', 'researchAndDevelopment', 'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome', 'interestIncome', 'interestExpense', 'nonInterestIncome', 'otherNonOperatingIncome', 'depreciation', 'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense', 'interestAndDebtExpense', 'netIncomeFromContinuingOperations', 'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome_x', 'reportedCurrency_y', 'totalAssets', 'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables', 'totalNonCurrentAssets', 'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill', 'i

In [5]:
asShare["daysToReport"].item(2000)

0.0

In [6]:
featureNames = ffd.getFeatureNames()
features = ffd.apply(pd.Timestamp(2020, 1, 1, tz = "UTC"), 1)

In [17]:
finPerDay = asset.shareprice.select(['Date', 'Close']) 
#add utc to every entry in the Date column
print(type(finPerDay))
finPerDay = finPerDay.with_columns(
    pl.col("Date").dt.replace_time_zone("UTC")
)

fin_quar = fin_quar.with_columns(
    [
        (pl.col(col) / pl.col("totalRevenue")).alias(col)
        for col in fin_quar.columns
        if fin_quar[col].dtype.is_numeric() and col != "totalRevenue"
    ]
)

for col in fin_quar.columns:
    print(f"col: {col}   item: {fin_quar[col].tail(1).item()}")

col: fiscalDateEnding   item: 2024-10-27 00:00:00+00:00
col: reportedDate   item: 2024-11-20 00:00:00+00:00
col: reportedEPS   item: 2.308876346844536e-11
col: estimatedEPS   item: 2.109343823043156e-11
col: surprise   item: 1.9953252380137965e-12
col: surprisePercentage   item: 2.6963970127130724e-10
col: reportTime   item: post-market
col: reportedCurrency_x   item: USD
col: grossProfit   item: 0.7455675275069836
col: totalRevenue   item: 35082000000.0
col: costOfRevenue   item: 0.25443247249301637
col: costofGoodsAndServicesSold   item: 0.009178496094863462
col: operatingIncome   item: 0.6233681090017673
col: sellingGeneralAndAdministrative   item: 0.025568667692833932
col: researchAndDevelopment   item: 0.09663075081238241
col: operatingExpenses   item: 0.12219941850521635
col: investmentIncomeNet   item: 0.013454193033464455
col: netInterestIncome   item: 0.011715409611766718
col: interestIncome   item: 0.013454193033464455
col: interestExpense   item: 0.0017387834216977367
col: n

In [24]:
fin_quar2 = fin_quar.rename({"fiscalDateEnding": "Date"})
fin_ann2 = fin_ann.rename({"fiscalDateEnding": "Date"})
# Perform asof joins to attach the closest (earlier) quarterly & annual rows
finPerDay = (
    finPerDay
    .join_asof(fin_quar2, on="Date", strategy="backward", suffix="_quar")
    .join_asof(fin_ann2, on="Date", strategy="backward", suffix="_ann")
)

In [ ]:
print(type(fin_quar["nonInterestIncome"].item(56)))
for col in fin_quar.columns:
        dtype = fin_quar[col].dtype
        # Check if the column is numeric
        if  dtype == fin_quar['nonInterestIncome'].dtype:
            min_val = fin_quar[col].min()
            max_val = fin_quar[col].max()
            print(f"{col} \n    Type: {dtype}, Min: {min_val}, Max: {max_val}")
        else:
            print(f"{col} \n    Type: {dtype}")